# Парсим TEI с помощью lxml

lxml - это питоновский модуль для работы с XML и HTML: с его помощью можно как создавать, так и парсить XML/HTML. В случае с TEI нас интересует XML. Давайте попробуем распарсить малый академический словарь (лежит в этом же репозитории, *MAS.tei*).

## etree

Для парсинга xml используется etree. Импортируем его и считаем им файл в tei.

In [2]:
from lxml import etree

In [3]:
# открываем файл со словарём
with open('MAS.tei') as f:
    text = f.read()
print(text[:200])

<xml>
	<fileDesc>
		<respStmt>
			<name>Вячеслав Иванов</name>
		</respStmt>
		<extent>1607</extent>
		<sourceDesc>
			<ref target="ссылка на источник в интернете">http://feb-web.ru/feb/mas/mas-abc/de


In [5]:
# создаём дерево etree из строки
root = etree.fromstring(text)

etree строит из xml/html докуменда дерева, где тэг со своим содержанием и свойствами -- узел, а вложенные в него тэги -- его дети. К детям узла можно обращаться по индексу, (а ещё их можно обходить циклом for).

От каждого узла можно вызвать .tag -- название тэга, а от некоторых -- text (вложенный текст), attrib (возвращает словарик с парами атрибут - его значение), tail (кусочек текста после закрывающего тэга).

In [5]:
root.tag

'xml'

In [7]:
root[0][0].tag

'respStmt'

In [9]:
print(root[0][0][0].tag)
print(root[0][0][0].text)

name
Вячеслав Иванов


In [14]:
root[0][0][0].tag

'name'

In [24]:
root[0][2][0].attrib

{'target': 'ссылка на источник в интернете'}

## XPath

! Вся описательная часть этого раздела скопирована из [туториала Эли Мустакимовой](https://github.com/elmiram/2017learnpython/blob/master/2%20%D0%A1%D0%B5%D0%BC%D0%B8%D0%BD%D0%B0%D1%80%20-%20lxml.ipynb). Спасибо Эле!

XPath - это способ представления пути к информации в структурированных документах типа XML/HTML. В XPath используются специальные выражения для того, чтобы выбирать какие-то узлы (элементы) в дереве XML. Есть [туториал по XPath на w3schools](https://www.w3schools.com/xml/xpath_nodes.asp), очень советую его почитать.

Элементы XPath выражения:
* tag - выбрать все узлы с названием tag
* / - указывает на узлы, которые являются прямыми потомками текущего
* // - указывает на узлы, которые являются потомками текущего (где-то внутри текущего узла, но необязательно его потомок)
* . - текущий узел
* .. - родитель текущего узла
* @ - указывает на атрибут
* text() - выбирает весь текст внутри узла (и внутри дочерних узлов тоже)

В XPath выражении могут быть предикаты, которые позволяют выбрать конкретный узел, предикаты пишутся в квадратных скобках. Например:
* //p[1] - выбрать первый элемент, который является ребенком узла с тэгом p
* //p[@style] - выбрать все узлы с тэгом p, у которых есть атрибут style
* //td[@border="1"] - выбрать все узлы с тэгом td, у которых атрибут border равен 1

Также есть специальные знаки, означающие "любой":
* \* - любой узел (например //\* - все элементы дерева, /body/\* - все дети тэга body)
* @ - любой атрибут (например `//p[@]` - все узлы с тэгом p, у которых есть хоть один какой-то атрибут)


### А тепеть давайте попробуем применить это к нашим данным:

In [28]:
# найдём все узлы с тэгом entry
entries = root.xpath('.//entry')

# получили массив тэгов
print(len(entries))
print(entries[0].tag)
print('\n\n***\n\n')

# преобразуем в строку и распечатаем
print(etree.tostring(entries[0], encoding='utf-8').decode())

1605
entry


***


<entry>
						<form>
							<orth>А1</orth>
						<gramGrp>
						</gramGrp>
						</form>
						<sense n="">
							<def>
								<text>нескл., ср. Название первой буквы русского алфавита.</text>
							</def>
						</sense>
						
						
							<sense n="">
								<def><text>От а до зет; от а до я — от начала до конца; всё целиком.</text></def>
							</sense>
						
					</entry>
					
					


In [30]:
# найдём все узлы со вторым смыслом
sec_sense = root.xpath('.//sense[@n="2"]')
print(etree.tostring(sec_sense[0], encoding='utf-8').decode())

<sense n="2">
							<def>
								<text>2. вопросительная. а) Употребляется как вопросительный отклик на обращение или при переспросе нерасслышанного. — Ридель! — повторил он громко. — Ридель! — А? Что таков? — проговорил я словно спросонья. Тургенев, Стук… стук… стук!  Вы представьте: в четверг я еду в деревню, вдруг свистят! Даже мне свистят, а? М. Горький, Враги. </text>
							</def>
						</sense>
					


### Задача: присвоить каждому тэгу entry аттрибут с порядковым номером

Решение:

In [33]:
# находим все тэги entry с помощью xpath
entries = root.xpath('.//entry')

# обходим их циклом
for i, entr in enumerate(entries):
    entr.set('id', str(i))

In [34]:
# смотрим, что получилось
print(etree.tostring(entries[6], encoding='utf-8').decode())

<entry id="6">
						<form>
							<orth>АБАЗИНСКИЙ</orth>
						<gramGrp>
							<paradigm>-ая, -ое</paradigm>
						</gramGrp>
						</form>
						<sense n="">
							<def>
								<text>Прил. к абазины. Абазинский язык.</text>
							</def>
						</sense>
					</entry>

					


## Выгружаем данные из etree дерева

In [13]:
# сохраняем whitespace-characters
text = etree.tostring(root[0][0], encoding='utf-8', pretty_print=True).decode()
print(text)

<respStmt>
			<name>Вячеслав Иванов</name>
		</respStmt>
		



In [14]:
# создаём ном новый красивый документ
from lxml import objectify
xml = objectify.fromstring(text)
text = etree.tostring(xml, encoding='utf-8', pretty_print=True).decode()
print(text)

<respStmt>
  <name>Вячеслав Иванов</name>
</respStmt>

